In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

### 1*.
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [4]:
def calc_logloss(y, y_pred, e=1e-8):
    y_pred[y_pred == 0] = e
    y_pred[y_pred == 1] = 1-e
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [6]:
### Logistic Regression

In [7]:
from sklearn.metrics import roc_auc_score

In [8]:
def eval_model(X, y, iterations, eta=1e-4, metric: callable = roc_auc_score):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    i, err = 0, 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= eta * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 1000) == 0:
        print(i, W, err, metric(y_true=y, y_score=sigmoid(W @ X)))
    return W

### 2.
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [9]:
for i in range(10):
    eta = 0.1**i
    print(f'\neta={eta:.0e}:')
    W = eval_model(X, y, iterations=5000, eta=eta)


eta=1e+00:
5000 [-142.20328298 -361.53825697 -299.34953546  510.6230347 ] 9.210340374463803 0.5

eta=1e-01:


<ipython-input-5-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))
<ipython-input-5-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))


5000 [-13.60328552 -36.79826351   3.14799581  52.5530304 ] 9.210340374463803 0.5

eta=1e-02:
5000

<ipython-input-5-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))


 [-0.91929485 -3.81662684 -4.37503497  6.61626835] 9.210340374463803 0.5

eta=1e-03:
5000 [ 0.35254905 -0.5001446   0.30755351  2.03038888] 105.28140486926335 0.5

eta=1e-04:


<ipython-input-5-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))


5000 [ 0.474378   -0.1676121   0.04121473  1.56894996] 2.849079424767922 0.6599999999999999

eta=1e-05:
5000 [ 0.49314177 -0.14151841 -0.00414786  1.52547952] 0.8471318040241776 0.6

eta=1e-06:
5000 [ 0.49571367 -0.13987934 -0.00243104  1.52220861] 0.6047007196670117 0.8400000000000001

eta=1e-07:
5000 [ 0.49646415 -0.1389143   0.41768854  1.52267986] 9.210340374463803 0.5

eta=1e-08:
5000 [ 0.49668915 -0.1383293   0.62468854  1.52299486] 9.210340374463803 0.5

eta=1e-09:
5000 [ 0.49671165 -0.1382708   0.64538854  1.52302636] 9.210340374463803 0.5


видим, что при eta=1e-6 logloss минимален, а метрика ROC-AUC максимальна для данного набора

In [10]:
W = eval_model(X, y, iterations=5000, eta=1e-6)

5000 [ 0.49571367 -0.13987934 -0.00243104  1.52220861] 0.6047007196670117 0.8400000000000001


### 3.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [11]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(W, X))

In [12]:
y_pred_proba = calc_pred_proba(W, X)
y_pred_proba

array([0.65982733, 0.54396557, 0.80800441, 0.60334475, 0.40022935,
       0.51826521, 0.67181125, 0.23173749, 0.68655806, 0.69621452])

### 4.
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [13]:
def calc_pred(W, X, threshold=0.5):
    y = calc_pred_proba(W, X)
    y[y > threshold] = 1
    y[y <= threshold] = 0
    return y

In [14]:
y_pred = calc_pred(W, X)
y_pred

array([1., 1., 1., 1., 0., 1., 1., 0., 1., 1.])

### 5.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [15]:
def error_matrix(y, y_pred):
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(y.shape[0]):
        if y[i] > y_pred[i]:
            FN += 1
        elif y[i] < y_pred[i]:
            FP += 1
        elif y[i] == 1:
            TP += 1
        else:
            TN += 1
    return np.array([[FN, FP], [TN, TP]])  # порядок такой, чтобы обращение было "говорящим": TN = array[1][0]

In [16]:
def measures(y, y_pred, betta=1):
    e_m = error_matrix(y, y_pred)
    TP = e_m[1][1]
    TN = e_m[1][0]
    FP = e_m[0][1]
    FN = e_m[0][0]
    accuracy = np.mean(y == y_pred)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    FPR = FP / (FP + TN)
    F_measure = (1 + betta**2) * precision * recall / (betta**2 * precision + recall)
    return accuracy, precision, recall, FPR, F_measure

In [17]:
accuracy, precision, recall, FPR, F_measure = measures(y, y_pred)
accuracy, precision, recall, FPR, F_measure

(0.5, 0.5, 0.8, 0.8, 0.6153846153846154)

### 6.
Могла ли модель переобучиться? Почему?

модель не переобучена, потому что accuracy по модулю далека от 1

### 7*.
Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [18]:
evals = dict()
evals['eval_model'] = accuracy, precision, recall, FPR, F_measure

#### $L_2$ регуляризация

In [19]:
def eval_model_l2(X, y, iterations, eta=1e-4, lmbda=1e-5, metric: callable = roc_auc_score):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    i, err = 0, 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= eta * (1/n * np.dot((y_pred - y), X.T))
        W -= lmbda * sum(W**2)
    if i % (iterations / 1000) == 0:
        print(i, W, err, metric(y_true=y, y_score=sigmoid(W @ X)))
    return W

In [20]:
W = eval_model_l2(X, y, iterations=5000, eta=1e-6)

5000 [ 0.37577599 -0.25943065 -0.00186926  1.40239255] 0.6120984762009825 0.8


In [21]:
y_pred = calc_pred(W, X)

In [22]:
accuracy, precision, recall, FPR, F_measure = measures(y, y_pred)
accuracy, precision, recall, FPR, F_measure

(0.5, 0.5, 0.8, 0.8, 0.6153846153846154)

In [23]:
evals['eval_model_l2'] = accuracy, precision, recall, FPR, F_measure

#### $L_1$ регуляризация

In [24]:
def eval_model_l1(X, y, iterations, eta=1e-4, lmbda=1e-5, metric: callable = roc_auc_score):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    i, err = 0, 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= eta * (1/n * np.dot((y_pred - y), X.T))
        W -= lmbda * sum(np.abs(W))
    if i % (iterations / 1000) == 0:
        print(i, W, err, metric(y_true=y, y_score=sigmoid(W @ X)))
    return W

In [25]:
W = eval_model_l1(X, y, iterations=5000, eta=1e-6)

5000 [ 0.38635453 -0.24888537 -0.00192084  1.41296025] 0.6111673613945812 0.8


In [26]:
y_pred = calc_pred(W, X)

In [27]:
accuracy, precision, recall, FPR, F_measure = measures(y, y_pred)
accuracy, precision, recall, FPR, F_measure

(0.5, 0.5, 0.8, 0.8, 0.6153846153846154)

In [28]:
evals['eval_model_l1'] = accuracy, precision, recall, FPR, F_measure

In [29]:
evals

{'eval_model': (0.5, 0.5, 0.8, 0.8, 0.6153846153846154),
 'eval_model_l2': (0.5, 0.5, 0.8, 0.8, 0.6153846153846154),
 'eval_model_l1': (0.5, 0.5, 0.8, 0.8, 0.6153846153846154)}

Регуляризация не сработала.
Видимо где-то накосячил, но не нашел где